In [1]:
import tensorflow.keras as keras
import tensorflow as tf

2024-01-23 11:50:00.506461: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 11:50:00.508086: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 11:50:00.542546: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-23 11:50:00.543882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-23 11:50:01.179018: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
def preprocess_data(X, Y):
    """
    Pre-processes the data for the model

        :param X: numpy.ndarray of shape (m, 32, 32, 3)
            containing the CIFAR 10 data, where m is the
            number of data points

        :param Y: numpy.ndarray of shape (m,) containing
            the CIFAR 10 labels for X

        :returns: X_p, Y_p
    """
    X_p = keras.applications.densenet.preprocess_input(X)

    # encode to one-hot
    Y_p = keras.utils.to_categorical(Y, 10)
    return X_p, Y_p


In [3]:
CALLBACKS = []
MODEL_PATH = 'cifar10.h5'
optimizer = keras.optimizers.Adam()

# load cifar 10
(train_images, train_labels), (val_images, val_labels) = keras.datasets.cifar10.load_data()

# extract a small sample for training and testing
n_train = 1000
train_images = train_images[:n_train]
train_labels = train_labels[:n_train]

n_val = 100
val_images = val_images[:n_val]
val_labels = val_labels[:n_val]

# pre-procces data
train_images, train_labels = preprocess_data(train_images, train_labels)
val_images, val_labels = preprocess_data(val_images, val_labels)

# input tensor
inputs = keras.Input(shape=(32, 32, 3))

# upscale layer
upscale = keras.layers.Lambda(lambda x: tf.image.resize_with_pad(x,
                                                                  160,
                                                                  160,
                                                                  method=tf.image.ResizeMethod.BILINEAR))(inputs)

In [4]:
# load base model
base_model = keras.applications.DenseNet121(include_top=False,
                                        weights='imagenet',
                                        input_tensor=upscale,
                                        input_shape=(160,160,3),
                                        pooling='max')

# set base model training off
base_model.trainable = False

# add top layers
out = base_model.output
out = keras.layers.Flatten()(out)
out = keras.layers.BatchNormalization()(out)
out = keras.layers.Dense(128, activation='relu')(out)
out = keras.layers.Dropout(0.3)(out)
out = keras.layers.BatchNormalization()(out)
out = keras.layers.Dense(64, activation='relu')(out)
out = keras.layers.Dropout(0.3)(out)
# out = keras.layers.BatchNormalization()(out)
# out = keras.layers.Dense(64, activation='relu')(out)
# out = keras.layers.Dropout(0.3)(out)
out = keras.layers.Dense(10, activation='softmax')(out)

# # callbacks
# CALLBACKS.append(keras.callbacks.ModelCheckpoint(filepath=MODEL_PATH,
#                                               monitor='val_acc',
#                                             #   save_best_only=True))

CALLBACKS.append(keras.callbacks.EarlyStopping(monitor='val_acc',
                                            verbose=1,
                                            patience=5))

# CALLBACKS.append(keras.callbacks.TensorBoard(log_dir='logs'))

# model compile
model = keras.models.Model(inputs=inputs, outputs=out)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])


# train
model.fit(x=train_images,
          y=train_labels,
          batch_size=32,
          epochs=10,
          callbacks=CALLBACKS,
          validation_data=(val_images, val_labels))

Epoch 1/10
32/32 [==============================] - 31s 792ms/step - loss: 1.9734 - accuracy: 0.3250 - val_loss: 1.9469 - val_accuracy: 0.4200
Epoch 2/10
32/32 [==============================] - 26s 803ms/step - loss: 1.0515 - accuracy: 0.6560 - val_loss: 1.1738 - val_accuracy: 0.6000
Epoch 3/10
32/32 [==============================] - 27s 854ms/step - loss: 0.7340 - accuracy: 0.7830 - val_loss: 0.8926 - val_accuracy: 0.6500
Epoch 4/10
32/32 [==============================] - 26s 813ms/step - loss: 0.5975 - accuracy: 0.8200 - val_loss: 0.6655 - val_accuracy: 0.7700
Epoch 5/10
32/32 [==============================] - 23s 732ms/step - loss: 0.4694 - accuracy: 0.8440 - val_loss: 0.5559 - val_accuracy: 0.8100
Epoch 6/10
32/32 [==============================] - 22s 700ms/step - loss: 0.3948 - accuracy: 0.8760 - val_loss: 0.5324 - val_accuracy: 0.7900
Epoch 7/10
32/32 [==============================] - 22s 684ms/step - loss: 0.3311 - accuracy: 0.9000 - val_loss: 0.4877 - val_accuracy: 0.8000

Epoch 20/20
50000/50000 [==============================] - 346s 7ms/sample - loss: 0.0614 - acc: 0.9809 - val_loss: 0.2649 - val_acc: 0.9272


<tensorflow.python.keras.callbacks.History at 0x7f0af909ff98>


In [6]:
model.evaluate(val_images, val_labels, batch_size=128, verbose=1)

1/1 [==============================] - 1s 1s/step - loss: 0.4900 - accuracy: 0.8100


[0.4900037348270416, 0.8100000023841858]